In [ ]:
from bs4 import BeautifulSoup
import requests

import unicodedata
from csv import writer
import re

import pandas as pd
import numpy as np

import json
from pyspark.sql import SparkSession
import os
import re 

#folium
import geopandas as gpd
import re
import folium

In [ ]:
def point_to_coor(df):

    df["lat"] = float(re.findall(r"\d+\.?\d*", df['geometry'][i])[0])
    df["lon"] = float(re.findall(r"\d+\.?\d*", df['geometry'][i])[1])
    return df


In [ ]:

output_dir = '../data/raw/'



In [ ]:
'''


property and elector count by postcode


'''

#url: https://discover.data.vic.gov.au/dataset/victorian-electors-by-locality-postcode-and-electorates 


columns = ['Locality Name','Post_x000D_\nCode','Property_x000D_\nCount','Elector_x000D_\nCount']

LocalityFinder_postcode = pd.read_excel('../data/raw/LocalityFinder_postcode.xlsx',
                                        sheet_name= 'Place_Names_Electronic',
                                        header=2)

LocalityFinder_postcode= LocalityFinder_postcode[columns].rename({'Post_x000D_\nCode': 'postcode',
                                                             'Locality Name':'suburb_name',
                                                             'Property_x000D_\nCount': 'property_count',
                                                            'Elector_x000D_\nCount':'elector_count'}, axis='columns')
property_and_elector = LocalityFinder_postcode.groupby('postcode').sum()

#output
filename = 'property_and_elector_by_postcode.csv'
output_dir_full = f'{output_dir}{filename}'
property_and_elector.to_csv(output_dir_full)

In [ ]:
"""
Download PTV zip file
1. open url: https://discover.data.vic.gov.au/dataset/ptv-metro-train-stations
2. select PTV shape file, add to cart and check out with email
3. open url in email recieved, download zip file
4. find folder with name 'PTV'
5. move folder under raw data folder


"""

In [ ]:
#PTV
ptv_df = gpd.read_file("../data/raw/PTV/PTV_METRO_TRAIN_STATION.shp")


#output
filename = 'train_staiton.csv'
output_dir_full = f'{output_dir}{filename}'
ptv_df.to_csv(output_dir_full)

In [ ]:
"""
Download FOI zip file
1. open url:https://datashare.maps.vic.gov.au/search?md=019d7631-1234-5112-9f21-8f7346647b61
2. add to cart and check out with email
3. open url in email recieved, download zip file
4. find folder with name 'VMFEAT'
5. move folder under raw data folder


"""

In [ ]:
# sf stands for shape file
sf = gpd.read_file("../data/raw/VMFEAT/FOI_POINT.shp")
VIC_FOI = sf.loc[sf['STATE']=='VIC']

In [ ]:
'''


hospital


'''
comunity_sector = ['communication service','NAME','geometry']
hospital = VIC_FOI.loc[VIC_FOI['FTYPE'] == 'hospital']
hospital_df = hospital[hospital_columns]
hospital_df = hospital_df.dropna(subset=['FEATSUBTYP','NAME','geometry'])


#output
filename = 'hospital.csv'
output_dir_full = f'{output_dir}{filename}'
hospital_df.to_csv(output_dir_full)

In [ ]:
'''

emergency services: police station, amubulance


'''
#filter out important emergency sevice
target_emergency_service= ['police station',
                           'ambulance station',
                           'fire station',
                           'fire station (forest industry)']
selected_columns = ['NAME_LABEL','FEATSUBTYP','geometry']

emergency_service_df = VIC_FOI.loc[VIC_FOI['FTYPE'] == 'emergency facility'][selected_columns]
emergency_service_df = emergency_service_df.loc[emergency_service_df['FEATSUBTYP']\
                                                .isin(target_emergency_service)]
emergency_service_df = emergency_service_df.dropna(subset=['NAME_LABEL','geometry'])


#output
filename = 'emergency_service.csv'
output_dir_full = f'{output_dir}{filename}'
emergency_service_df.to_csv(output_dir_full)




In [ ]:
'''


public service: 
swimming pool,libary museum,art gallery


'''
service_type= ['sport facility',
               'cultural centre',
               'community space']

public_service = VIC_FOI.loc[VIC_FOI['FTYPE']\
                             .isin(service_type)]
target_service = ['camp ground',
                  'swimming pool',
                  'library',
                  'museum',
                  'art gallery',
                  'aquarium',
                  'observatory']
selected_columns = ['FTYPE','FEATSUBTYP','NAME','geometry'] 
public_service = public_service.loc[public_service['FEATSUBTYP']\
                             .isin(target_service)][selected_columns]
public_service_df = public_service.dropna(subset=['NAME','geometry'])


#output
filename = 'public_service.csv'
output_dir_full = f'{output_dir}{filename}'
public_service_df.to_csv(output_dir_full)




In [ ]:
'''


care facility (child,disability,aged)



'''

selected_columns = ['NAME_LABEL','FEATSUBTYP','geometry']
care_facility_df = VIC_FOI.loc[VIC_FOI['FTYPE'] =='care facility'][selected_columns]

#output
filename = 'care_facility.csv'
output_dir_full = f'{output_dir}{filename}'
care_facility_df.to_csv(output_dir_full)


